# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1QHOQt1J7d5Z4sBFKL9GN699OcPh2aVaP?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [19]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys 

nav = webdriver.Chrome()

#pesquisar cotação do dolar
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_dolar = nav.find_element_by_xpath('/html/body/div[7]/div/div[9]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[1]/div/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)

#pesquisar cotacao do euro
nav.get("https://www.google.com/")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
nav.find_element_by_xpath('/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = nav.find_element_by_xpath('/html/body/div[7]/div/div[9]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div/div/div/div[1]/div/div[1]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#pesquisar cotação ouro
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0] #variavel que armazéna que a página abaixo se trata da aba "original"
nav.find_element_by_xpath('/html/body/div[15]/div[2]/div/table[2]/tbody/tr[2]/td[2]/a/img').click()

#pegar cotacao do ouro 
nav.get("https://www.melhorcambio.com/")
aba_original = nav.window_handles[0] #variavel que armazéna que a página abaixo se trata da aba "original"
nav.find_element_by_xpath('/html/body/div[15]/div[2]/div/table[2]/tbody/tr[2]/td[2]/a/img')
aba_nova = nav.window_handles[1]
nav.switch_to.window(aba_nova)
cotacao_ouro = nav.find_element_by_id('comercial').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

nav.quit()

5.4399999999999995
6.553948800000001
313.44


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [22]:
import pandas as pd 
produtos_df = pd.read_excel('Produtos.xlsx')
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [23]:
produtos_df.loc[produtos_df['Moeda']=="Dólar", "Cotação"] = float(cotacao_dolar)
produtos_df.loc[produtos_df['Moeda']=="Euro", "Cotação"] = float(cotacao_euro)
produtos_df.loc[produtos_df['Moeda']=="Ouro", "Cotação"] = float(cotacao_ouro)
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.440000,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6.553949,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5.440000,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5.440000,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6.553949,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5.440000,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,313.440000,7000.00,1.15,8050.000


In [24]:
produtos_df['Preço Base Reais'] = produtos_df['Cotação'] * produtos_df['Preço Base Original']
produtos_df['Preço Final'] = produtos_df['Ajuste'] * produtos_df['Preço Base Reais']
display(produtos_df)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Ajuste,Preço Final
0,Câmera Canon,999.99,Dólar,5.440000,5439.9456,1.40,7615.92384
1,Carro Renault,4500.00,Euro,6.553949,29492.7696,2.00,58985.53920
2,Notebook Dell,899.99,Dólar,5.440000,4895.9456,1.70,8323.10752
3,IPhone,799.00,Dólar,5.440000,4346.5600,1.70,7389.15200
4,Carro Fiat,3000.00,Euro,6.553949,19661.8464,1.90,37357.50816
5,Celular Xiaomi,480.48,Dólar,5.440000,2613.8112,2.00,5227.62240
6,Joia 20g,20.00,Ouro,313.440000,6268.8000,1.15,7209.12000


### Agora vamos exportar a nova base de preços atualizada

In [18]:
produtos_df.to_excel("Produtos Atualizado.xlsx", index=False)